In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_data = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x



class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = models.alexnet(pretrained=False).features
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


models_list = {
    "LeNet-5": LeNet5,
    "AlexNet": AlexNet,
    "VGG": models.vgg16,
    "ResNet-18": models.resnet18,
    "ResNet-50": models.resnet50,
    "ResNet-101": models.resnet101
}

def load_pretrained_weights(model_name):
    if model_name == "LeNet-5":
        return LeNet5().to(device)
    elif model_name == "AlexNet":
        model = AlexNet(num_classes=10)
        pretrained_dict = models.alexnet(pretrained=True).state_dict()
        model_dict = model.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        # Update the state dict
        model_dict.update(pretrained_dict)
        # Load the updated state dict
        model.load_state_dict(model_dict)
        return model.to(device)
    else:
        model = models_list[model_name](pretrained=True)
        # Replace the last layer with custom output layer to match SVHN
        if "ResNet" in model_name:
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, 10)
        else:
            num_ftrs = model.classifier[-1].in_features
            model.classifier[-1] = nn.Linear(num_ftrs, 10)
        return model.to(device)


def train_model(model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct / total

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {100*train_accuracy:.2f}%")



def test_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_accuracy = correct / total
    print(f"Test Accuracy: {100*test_accuracy:.2f}%")




my_list = []
for model_name in models_list.keys():
    my_list.append(model_name)


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [5]:
#LeNet-5 Model

model = load_pretrained_weights(my_list[0])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[0]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[0]}...")
test_model(model)
print()

Training LeNet-5...
Epoch 1/5, Train Loss: 2.2502, Train Accuracy: 17.86%
Epoch 2/5, Train Loss: 2.2330, Train Accuracy: 18.92%
Epoch 3/5, Train Loss: 2.2164, Train Accuracy: 18.96%
Epoch 4/5, Train Loss: 1.8958, Train Accuracy: 34.04%
Epoch 5/5, Train Loss: 1.0928, Train Accuracy: 66.39%
Testing LeNet-5...
Test Accuracy: 76.62%



In [8]:
#VGG model
model = load_pretrained_weights(my_list[2])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[2]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[2]}...")
test_model(model)
print()

Training VGG...
Epoch 1/5, Train Loss: 0.5971, Train Accuracy: 80.54%
Epoch 2/5, Train Loss: 0.2786, Train Accuracy: 91.75%
Epoch 3/5, Train Loss: 0.2155, Train Accuracy: 93.78%
Epoch 4/5, Train Loss: 0.1785, Train Accuracy: 94.85%
Epoch 5/5, Train Loss: 0.1498, Train Accuracy: 95.87%
Testing VGG...
Test Accuracy: 94.63%



In [12]:
#ResNet 18

model = load_pretrained_weights(my_list[3])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[3]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[3]}...")
test_model(model)
print()


#ResNet 50

model = load_pretrained_weights(my_list[4])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[4]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[4]}...")
test_model(model)
print()

#ResNet 100

model = load_pretrained_weights(my_list[4])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[5]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[5]}...")
test_model(model)
print()

Training ResNet-18...
Epoch 1/5, Train Loss: 0.7922, Train Accuracy: 73.73%
Epoch 2/5, Train Loss: 0.3456, Train Accuracy: 89.25%
Epoch 3/5, Train Loss: 0.2587, Train Accuracy: 92.04%
Epoch 4/5, Train Loss: 0.2089, Train Accuracy: 93.60%
Epoch 5/5, Train Loss: 0.1687, Train Accuracy: 94.81%
Testing ResNet-18...
Test Accuracy: 91.78%

Training ResNet-50...
Epoch 1/5, Train Loss: 0.7152, Train Accuracy: 76.28%
Epoch 2/5, Train Loss: 0.2783, Train Accuracy: 91.56%
Epoch 3/5, Train Loss: 0.1920, Train Accuracy: 94.22%
Epoch 4/5, Train Loss: 0.1412, Train Accuracy: 95.73%
Epoch 5/5, Train Loss: 0.1063, Train Accuracy: 96.79%
Testing ResNet-50...
Test Accuracy: 93.00%

Training ResNet-101...
Epoch 1/5, Train Loss: 0.7104, Train Accuracy: 76.56%
Epoch 2/5, Train Loss: 0.2764, Train Accuracy: 91.65%
Epoch 3/5, Train Loss: 0.1954, Train Accuracy: 94.09%
Epoch 4/5, Train Loss: 0.1438, Train Accuracy: 95.65%
Epoch 5/5, Train Loss: 0.1079, Train Accuracy: 96.80%
Testing ResNet-101...
Test Accuracy

In [ ]:
model = load_pretrained_weights(my_list[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[1]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[1]}...")
test_model(model)
print()